In [ ]:
import glob
import nltk
import pandas as pd
from sklearn import svm
from sklearn import metrics
from sklearn import naive_bayes
from gensim.models import KeyedVectors
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 

<div class="alert alert-block alert-warning">
    1. Reading & PreProcessing Training & Testing Data
</div>

In [10]:
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
li = []
Cols = ['ID','Label', 'Tweet']
path = r'C:\\Users\\khled\\Downloads\\Model\\'
all_files = glob.glob(path + "/twitter-201*.txt")
#Reading all files:
for file in all_files:
    df = pd.read_csv(file, names=Cols, delimiter='\t')
    li.append(df)

DF = pd.concat(li, axis=0, ignore_index=True)
Data = DF.drop('ID', axis=1)
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()
stop = stopwords.words('english')
Data['Tweet'] = Data['Tweet'].replace({r'\\':'',r'\'':'',r'\,':'','&':'',r'\"':'','!':'','\.':'','u2019':'\'','u002c':',','(@[A-Za-z0-9]+)|(\w+:\/\/\S+)':''}, regex=True)
Data['Tweet'] = [ tweet.casefold() for tweet in Data['Tweet'] ]
Data['Tweet'] = Data['Tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
Data['Tweet'] = Data['Tweet'].apply(lambda x: ' '.join([porter.stem(word) for word in x.split()]))
def cond(i):
    if     i == 0: return 'neutral'
    elif   i == 1: return 'positive'
    elif   i == 2: return 'negative'
    return i
#Reading Test and Test Labels for evaluating results
Test = pd.read_csv("C:\\Users\\khled\\Downloads\\Model\\test.csv")
test_labels = pd.read_csv("C:\\Users\\khled\\Downloads\\Model\\answer_key.csv")
test_labels['vlabels'] = [cond(label) for label in test_labels['label']]
Test['tweet'] = Test['tweet'].replace({r'\\':'',r'\'':'',r'\,':'','&':'',r'\"':'','!':'','\.':'','u2019':'\'','u002c':',','(@[A-Za-z0-9]+)|(\w+:\/\/\S+)':''}, regex=True)
Test['tweet'] = [ tweet.casefold() for tweet in Test['tweet'] ]
Test['tweet'] = Test['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
Test['tweet'] = Test['tweet'].apply(lambda x: ' '.join([porter.stem(word) for word in x.split()]))
print(len(Data['Tweet']))

16041


<div class="alert alert-block alert-warning">
    2. Transform Training & Testing Data to vectors
</div>

In [29]:
#Transform tweets to array
Tweets_wv = []
for i in range(0,16041):
    F = 1
    Sentince = [0]*300
    for x in Data['Tweet'][i].split():
        found = x in model.wv
        if found:
            F += 1
            Sentince = Sentince + model[x]
    Sentince = [ e / F-1 for e in Sentince ]
    Tweets_wv.append(Sentince)

print(len(Tweets_wv[0]))

C:\Users\khled\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


300


In [34]:
Test_wv = []
for i in range(0,3096):
    F = 1
    Sentince = [0]*300
    for x in Test['tweet'][i].split():
        found = x in model.wv
        if found:
            F += 1
            Sentince = Sentince + model[x]
    Sentince = [ e / F-1 for e in Sentince ]
    Test_wv.append(Sentince)

print(len(Test_wv[0]))

C:\Users\khled\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


300


<div class="alert alert-block alert-warning">
    3. Normalizing the vectors
</div>

In [35]:
Scaler = MinMaxScaler()
Tweets_wv = Scaler.fit_transform(Tweets_wv)
Test_wv = Scaler.fit_transform(Test_wv)
#print(Tweets_wv[0])

<div class="alert alert-block alert-warning">
    4. Testing
</div>

In [36]:
#Naïve Bayes Multinomial
Clf = naive_bayes.MultinomialNB()
Clf.fit(Tweets_wv,Data['Label'])
Pred = Clf.predict(Test_wv)
acc = metrics.accuracy_score(test_labels['vlabels'],Pred)
print ('accuracy = '+str(acc*100)+'%')
print (metrics.classification_report(test_labels['vlabels'],Pred))

accuracy = 56.16925064599483%
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00       460
     neutral       0.55      0.77      0.64      1374
    positive       0.58      0.54      0.56      1262

   micro avg       0.56      0.56      0.56      3096
   macro avg       0.38      0.44      0.40      3096
weighted avg       0.48      0.56      0.51      3096



C:\Users\khled\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [38]:
#SVM (Linear Kernel)
Clf = svm.SVC(kernel='linear', C=1)
Clf.fit(Tweets_wv,Data['Label'])
Pred = Clf.predict(Test_wv)
acc = metrics.accuracy_score(test_labels['vlabels'],Pred)
print ('accuracy = '+str(acc*100)+'%')
print (metrics.classification_report(test_labels['vlabels'],Pred))

accuracy = 54.78036175710594%
              precision    recall  f1-score   support

    negative       0.35      0.67      0.46       460
     neutral       0.59      0.71      0.64      1374
    positive       0.74      0.33      0.46      1262

   micro avg       0.55      0.55      0.55      3096
   macro avg       0.56      0.57      0.52      3096
weighted avg       0.61      0.55      0.54      3096



In [39]:
#Logistic Regression
Clf = LogisticRegression()
Clf.fit(Tweets_wv,Data['Label'])
Pred = Clf.predict(Test_wv)
acc = metrics.accuracy_score(test_labels['vlabels'],Pred)
print ('accuracy = '+str(acc*100)+'%')
print (metrics.classification_report(test_labels['vlabels'],Pred))

C:\Users\khled\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\khled\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


accuracy = 57.428940568475454%
              precision    recall  f1-score   support

    negative       0.40      0.58      0.47       460
     neutral       0.58      0.76      0.66      1374
    positive       0.74      0.37      0.49      1262

   micro avg       0.57      0.57      0.57      3096
   macro avg       0.57      0.57      0.54      3096
weighted avg       0.62      0.57      0.56      3096

